In [1]:
import pandas as pd
import os
import torch
import numpy as np
from tqdm.notebook import tqdm
import random

In [2]:
import gc

In [3]:
from Config import *
import TextDataFrame as tdf
import utils
import Embedding as eb

import imp
imp.reload(tdf)
imp.reload(utils)
imp.reload(eb)

<module 'Embedding' from '/home/adrian/Projects/Competition/kaggle-toxic-comments-2021/Embedding.py'>

In [4]:
utils.set_seed()

### 1. Read Data

In [5]:
!ls -ltr {DATA_PATH}

total 20
drwxr-xr-x 2 adrian adrian 4096 Dec 18 09:29 toxic-comment-classification-challenge
drwxr-xr-x 2 adrian adrian 4096 Dec 18 09:30 multilingual-toxic-comment-classification
drwxr-xr-x 2 adrian adrian 4096 Dec 18 09:30 toxic-severity-rating
drwxr-xr-x 2 adrian adrian 4096 Dec 18 09:31 unintended-bias-in-toxicity-classification
drwxr-xr-x 4 adrian adrian 4096 Dec 26 19:21 ruddit-jigsaw-dataset


In [6]:
df_processor = tdf.DatasetProcessor(DEFAULT_CLEAN_PROCEDURE)

In [7]:
data_dict = {}

#### 1.1 toxic-comment-classification-challenge

In [8]:
!ls -ltr {DATA_PATH}/toxic-comment-classification-challenge

total 137136
-rw-r--r-- 1 adrian adrian 68802655 Jan 18  2018 train.csv
-rw-r--r-- 1 adrian adrian 60354593 Jan 18  2018 test.csv
-rw-r--r-- 1 adrian adrian  6279782 Jun 19  2018 sample_submission.csv
-rw-r--r-- 1 adrian adrian  4976930 Jun 19  2018 test_labels.csv


In [9]:
df_train = pd.read_csv(os.path.join(DATA_PATH,"toxic-comment-classification-challenge/train.csv"))

In [10]:
df_train['comment_text'].duplicated().sum()

0

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [12]:
df_train.corr()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
toxic,1.000000,0.308619,0.676515,0.157058,0.647518,0.266009
severe_toxic,0.308619,1.000000,0.403014,0.123601,0.375807,0.201600
obscene,0.676515,0.403014,1.000000,0.141179,0.741272,0.286867
threat,0.157058,0.123601,0.141179,1.000000,0.150022,0.115128
insult,0.647518,0.375807,0.741272,0.150022,1.000000,0.337736
identity_hate,0.266009,0.201600,0.286867,0.115128,0.337736,1.000000


In [13]:
data_dict['toxic-comment-classification-challenge'] = df_train

#### 1.2 multilingual-toxic-comment-classification

In [14]:
!ls -ltr {DATA_PATH}/multilingual-toxic-comment-classification

total 4913992
-rw-r--r-- 1 adrian adrian  406706887 Sep 28 16:35 jigsaw-toxic-comment-train-processed-seqlen128.csv
-rw-r--r-- 1 adrian adrian   95538001 Sep 28 16:36 jigsaw-toxic-comment-train.csv
-rw-r--r-- 1 adrian adrian 3498299507 Sep 28 16:36 jigsaw-unintended-bias-train-processed-seqlen128.csv
-rw-r--r-- 1 adrian adrian  860317123 Sep 28 16:39 jigsaw-unintended-bias-train.csv
-rw-r--r-- 1 adrian adrian  122946125 Sep 28 16:40 test-processed-seqlen128.csv
-rw-r--r-- 1 adrian adrian     627019 Sep 28 16:40 sample_submission.csv
-rw-r--r-- 1 adrian adrian   28783206 Sep 28 16:40 test.csv
-rw-r--r-- 1 adrian adrian   14997354 Sep 28 16:40 validation-processed-seqlen128.csv
-rw-r--r-- 1 adrian adrian     499395 Sep 28 16:40 test_labels.csv
-rw-r--r-- 1 adrian adrian    3178555 Sep 28 16:40 validation.csv


In [15]:
!cat {DATA_PATH}/multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv | head -5

id,comment_text,toxic,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
59848,"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",0.0,0.0,0.0,0.0,0.0,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:41.987077+00,2,,2006,rejected,0,0,0,0,0,0.0,0,4
59849,"Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",0.0,0.0,0.0,0.0,0.0,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:42.870083+00,2,,2006,rejected,0,0,0,0,

In [16]:
df_train_1 = pd.read_csv(
                    os.path.join(DATA_PATH,"multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv"),
                    usecols = ['id', 'comment_text', 'toxic', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
                    )\
                .rename(
                    columns = {'severe_toxicity':'severe_toxic','identity_attack':'identity_hate'
                    })
df_train_2 = pd.read_csv(os.path.join(DATA_PATH,"multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv"))
df_train = pd.concat([df_train_1, df_train_2], axis = 0)
df_train = df_train.groupby('comment_text',as_index = False).mean()

del df_train_1, df_train_2
gc.collect()

8

In [17]:
df_train.groupby?

In [18]:
df_train['comment_text'].duplicated().sum()

0

In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2099954 entries, 0 to 2099953
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   comment_text   object 
 1   toxic          float64
 2   severe_toxic   float64
 3   obscene        float64
 4   identity_hate  float64
 5   insult         float64
 6   threat         float64
dtypes: float64(6), object(1)
memory usage: 128.2+ MB


In [20]:
df_train.corr()

,toxic,severe_toxic,obscene,identity_hate,insult,threat
toxic,1.000000,0.310023,0.519485,0.418751,0.877831,0.265656
severe_toxic,0.310023,1.000000,0.362707,0.202598,0.283395,0.186775
obscene,0.519485,0.362707,1.000000,0.154304,0.465149,0.087991
identity_hate,0.418751,0.202598,0.154304,1.000000,0.318431,0.143788
insult,0.877831,0.283395,0.465149,0.318431,1.000000,0.116596
threat,0.265656,0.186775,0.087991,0.143788,0.116596,1.000000


In [21]:
data_dict["multilingual-toxic-comment-classification"] = df_train

#### 1.3 unintended-bias-in-toxicity-classification

In [22]:
!ls -ltr {DATA_PATH}/unintended-bias-in-toxicity-classification

total 2320180
-rw-r--r-- 1 adrian adrian 916052376 Jul 16  2020 all_data.csv
-rw-r--r-- 1 adrian adrian 106388260 Jul 16  2020 identity_individual_annotations.csv
-rw-r--r-- 1 adrian adrian  29925195 Jul 16  2020 test.csv
-rw-r--r-- 1 adrian adrian   1167854 Jul 16  2020 sample_submission.csv
-rw-r--r-- 1 adrian adrian  44107259 Jul 16  2020 test_private_expanded.csv
-rw-r--r-- 1 adrian adrian  44336246 Jul 16  2020 test_public_expanded.csv
-rw-r--r-- 1 adrian adrian 417648663 Jul 16  2020 toxicity_individual_annotations.csv
-rw-r--r-- 1 adrian adrian 816211476 Jul 16  2020 train.csv


In [23]:
!cat {DATA_PATH}/unintended-bias-in-toxicity-classification/all_data.csv | head -3

id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,male,female,transgender,other_gender,heterosexual,homosexual_gay_or_lesbian,bisexual,other_sexual_orientation,christian,jewish,muslim,hindu,buddhist,atheist,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
1083994,He got his money... now he lies in wait till after the election in 2 yrs.... dirty politicians need to be afraid of Tar and feathers again... but they aren't and so the people get screwed.,train,2017-03-06 15:21:53.675241+00,21,,317120,approved,0,0,0,2,0,0.373134328358209,0.04477611940298507,0.08955223880597014,0.01492537313432836,0.0,0.34328358208955223,0.01492537313432836,,,,,,,,,,,,,,,,,,,,,,,,,0,67
650904,Mad dog will surely

In [24]:
df_train = pd.read_csv(
                    os.path.join(DATA_PATH,"unintended-bias-in-toxicity-classification/all_data.csv"),
                    usecols = ["id","comment_text","toxicity","severe_toxicity","obscene",
                               "sexual_explicit","identity_attack","insult","threat"]
                    )

In [25]:
df_train['comment_text'].duplicated().sum()

27600

In [26]:
df_train = df_train.groupby('comment_text',as_index = False).mean()

In [27]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971915 entries, 0 to 1971914
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   comment_text     object 
 1   id               float64
 2   toxicity         float64
 3   severe_toxicity  float64
 4   obscene          float64
 5   sexual_explicit  float64
 6   identity_attack  float64
 7   insult           float64
 8   threat           float64
dtypes: float64(8), object(1)
memory usage: 150.4+ MB


In [28]:
df_train.corr()

,id,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat
id,1.000000,0.004453,0.001063,-0.005217,-0.005064,0.010668,0.003423,0.001537
toxicity,0.004453,1.000000,0.393375,0.492872,0.252374,0.449990,0.928195,0.287775
severe_toxicity,0.001063,0.393375,1.000000,0.340632,0.196936,0.256811,0.324829,0.277658
obscene,-0.005217,0.492872,0.340632,1.000000,0.333652,0.124341,0.426581,0.091125
sexual_explicit,-0.005064,0.252374,0.196936,0.333652,1.000000,0.117696,0.145004,0.088448
identity_attack,0.010668,0.449990,0.256811,0.124341,0.117696,1.000000,0.310767,0.146268
insult,0.003423,0.928195,0.324829,0.426581,0.145004,0.310767,1.000000,0.108939
threat,0.001537,0.287775,0.277658,0.091125,0.088448,0.146268,0.108939,1.000000


In [29]:
data_dict["unintended-bias-in-toxicity-classification"] = df_train

#### 1.4 ruddit-jigsaw-dataset

In [30]:
!ls -ltr {DATA_PATH}/ruddit-jigsaw-dataset/Dataset

total 32512
-rw-r--r-- 1 adrian adrian  4077200 Dec  1 11:48 Ruddit_individual_annotations.csv
-rw-r--r-- 1 adrian adrian   128343 Dec  1 11:48 Ruddit.csv
-rw-r--r-- 1 adrian adrian      265 Dec  1 11:48 ReadMe.md
-rw-r--r-- 1 adrian adrian   115216 Dec  1 11:48 sample_input_file.csv
-rw-r--r-- 1 adrian adrian  1835021 Dec  1 11:48 ruddit_with_text.csv
-rw-r--r-- 1 adrian adrian       52 Dec  1 11:48 post_with_issues.csv
-rw-r--r-- 1 adrian adrian 15674990 Dec  1 11:48 node_dictionary.npy
-rw-r--r-- 1 adrian adrian     1354 Dec  1 11:48 load_node_dictionary.py
-rw-r--r-- 1 adrian adrian      454 Dec  1 11:48 identityterms_group.txt
-rw-r--r-- 1 adrian adrian     8754 Dec  1 11:48 create_dataset_variants.py
-rw-r--r-- 1 adrian adrian 11419592 Dec  1 11:48 Thread_structure.txt


In [31]:
!cat {DATA_PATH}/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv | head -3

post_id,comment_id,txt,url,offensiveness_score
42g75o,cza1q49,"> The difference in average earnings between men and women can be explained by taking into account relevant factors.

cat: write error: Broken pipe


In [32]:
df_train = pd.read_csv(
                    os.path.join(DATA_PATH,"ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv"),
                    usecols = ['comment_id', 'txt','offensiveness_score']
                    )

In [33]:
df_train['txt'].duplicated().sum()

126

In [34]:
df_train = df_train.groupby('txt',as_index = False).mean()

In [35]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5712 entries, 0 to 5711
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   txt                  5712 non-null   object 
 1   offensiveness_score  5712 non-null   float64
dtypes: float64(1), object(1)
memory usage: 133.9+ KB


In [36]:
df_train.corr()

,offensiveness_score
offensiveness_score,1.0


In [37]:
data_dict["ruddit-jigsaw-dataset"] = df_train

#### 1.5 Data Integration

In [38]:
rename_columns = {
    "comment_text":"text", "txt":"text",
    "comment_id":"id",
    "identity_attack":'identity_hate',
    'toxicity':'toxic',
    'severe_toxicity':'severe_toxic'
}
for value in data_dict.values(): value.rename(columns = rename_columns, inplace = True)
for value in data_dict.values(): 
    if 'id' in value.columns: value.drop('id', axis = 1, inplace = True)

In [39]:
for value in data_dict.values(): 
    value = value[['text'] + list(np.sort([x for x in value.columns if x != 'text']))]

In [40]:
dataset_key_map_dict = {key:idx for idx, key in enumerate(data_dict.keys())}

In [41]:
for key,value in data_dict.items(): 
    value['source'] = dataset_key_map_dict.get(key)

In [42]:
{key:value.columns for key, value in data_dict.items()}

{'toxic-comment-classification-challenge': Index(['text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
        'identity_hate', 'source'],
       dtype='object'),
 'multilingual-toxic-comment-classification': Index(['text', 'toxic', 'severe_toxic', 'obscene', 'identity_hate', 'insult',
        'threat', 'source'],
       dtype='object'),
 'unintended-bias-in-toxicity-classification': Index(['text', 'toxic', 'severe_toxic', 'obscene', 'sexual_explicit',
        'identity_hate', 'insult', 'threat', 'source'],
       dtype='object'),
 'ruddit-jigsaw-dataset': Index(['text', 'offensiveness_score', 'source'], dtype='object')}

In [43]:
# col = 'identity_hate'
# pd.concat([data_dict[dataset][col].describe().to_frame(dataset) for dataset in shared_dataset], axis = 1).T.round(3)

NameError: name 'shared_dataset' is not defined

In [44]:
data = pd.concat(data_dict.values(), axis = 0)
del data_dict
gc.collect()

150

In [48]:
unique_text = data['text'].unique()
# text_series = df_processor.clean_text(pd.DataFrame(unique_text)[0])
#np.save('all_text_embedding.npy', text_series)

In [49]:
text_series = np.load('all_text_embedding.npy', allow_pickle = True)
text_series = pd.DataFrame(text_series, index = unique_text)
text_series = text_series[0].to_dict()

In [50]:
data['text'] = data['text'].progress_apply(lambda x:text_series[x])
data = data.groupby('text', as_index = False).mean()
data = data.loc[data['text'].map(lambda x:len(x)>1)]

  0%|          | 0/4237152 [00:00<?, ?it/s]

In [54]:
weights_target_dict = {'obscene': 1, 'toxic': 1, 'threat': 1,  'insult': 1, 'severe_toxic': 1, 'identity_hate': 2}
text_col = 'comment_text'
data['y'] = df_processor.set_target(data, average_weights_dict = weights_target_dict)

In [55]:
data['y'] = (data['y'] - data['y'].min()) / (data['y'].max() - data['y'].min())
data['offensiveness_score'] = (data['offensiveness_score'] - data['offensiveness_score'].min()) / (data['offensiveness_score'].max() - data['offensiveness_score'].min())
data.loc[data['toxic'].isnull(),'y'] = data.loc[data['toxic'].isnull(),'offensiveness_score']

### Tokenizer

In [58]:
text_col = 'text'

In [59]:
tokenizer = eb.Embedding()
tokenizer.fit(data[text_col])

In [60]:
data_new = utils.sample_binary(data, 'y')
y = data_new['y']

In [ ]:
from sklearn.linear_model import Ridge

X_TFIDF = tokenizer.transform('TFIDF', data_new[text_col])
model_ridge_TFIDF = Ridge(alpha=0.5)
model_ridge_TFIDF.fit(X_TFIDF, y)

In [ ]:
# X_BERT = tokenizer.transform('BERT', data_new[text_col], 100)
# np.save('BERT_all_text_embedding.npy',X_BERT)

# model_ridge_BERT = Ridge(alpha=0.5)

# model_ridge_BERT.fit(X_BERT, y)

In [ ]:
def predict(text_series, text_cleaner_func, tokenizer_transform_func, predict_func,
           ):
    cleaned_text = text_cleaner_func(text_series)
    cleaned_text = tokenizer_transform_func(cleaned_text)
    probability =  predict_func(cleaned_text)
    return probability
def evaluate(text_cleaner_func, tokenizer_transform_func, predict_func):

    df_val = pd.read_csv(os.path.join(DATA_PATH,"toxic-severity-rating/validation_data.csv"))

    p1 = predict(df_val['less_toxic'], 
                 text_cleaner_func, 
                 tokenizer_transform_func,
                 predict_func
                )

    p2 = predict(df_val['more_toxic'], 
                 text_cleaner_func, 
                 tokenizer_transform_func,
                 predict_func
                )

    return {"% Correct":(p1<p2).mean(), "% Equal":(p1==p2).mean()}

In [ ]:
df_val = pd.read_csv(os.path.join(DATA_PATH,"toxic-severity-rating/validation_data.csv"))

In [ ]:
text_series = df_val['less_toxic']
cleaned_text =  df_processor.clean_text(text_series)
cleaned_text = tokenizer.transform('TFIDF', cleaned_text)
p1 = model_ridge_TFIDF.predict(cleaned_text)

text_series = df_val['more_toxic']
cleaned_text =  df_processor.clean_text(text_series)
cleaned_text = tokenizer.transform('TFIDF', cleaned_text)
p2 = model_ridge_TFIDF.predict(cleaned_text)

In [ ]:
{"% Correct":(p1<p2).mean(), "% Equal":(p1==p2).mean()}

In [ ]:
text_series = df_val['less_toxic']
cleaned_text =  df_processor.clean_text(text_series)
cleaned_text = batch_transform(tokenizer, 'BERT', cleaned_text, 100)
p1_BERT = model_ridge_BERT.predict(cleaned_text)

text_series = df_val['more_toxic']
cleaned_text =  df_processor.clean_text(text_series)
cleaned_text = batch_transform(tokenizer, 'BERT', cleaned_text, 100)
p2_BERT = model_ridge_BERT.predict(cleaned_text)

In [ ]:
{"% Correct":(p1_BERT<p2_BERT).mean(), "% Equal":(p1_BERT==p2_BERT).mean()}